In [42]:
import numpy as np

In [43]:
def get_ground_state(num_qubits):
    # returns vector of size 2**num_qubits with all zeroes except first element which is 1
    ground_state = np.zeros((2**num_qubits,1))
    ground_state[0][0] = 1
    
    return ground_state

In [44]:
def get_operator(total_qubits, gate_unitary, target_qubits):
    # returns unitary operator of size 2**n x 2**n for given gate and target qubits    
    # gate_unitary, target_qubits should be numpy arraytype    
    # supports 1 qubit unitaries (U) and 2 qubit CU type gates    
    # CU = | I 0 |
    #      | 0 U |
    
    # For U  -> target_qubits = [target]
    # For CU -> target_qubits = [control, target]
    
    is_control_gate = (target_qubits.size == 2)
    
    if is_control_gate:
        unitary = gate_unitary[2:4,2:4]   # extract the unitary applied by the CU gate
    else:
        unitary = gate_unitary
        
    # Define 2x2 Identity

    I = np.identity(2)    
        
    if is_control_gate:
        # CU gate
        
        # Define projection operator |0><0|

        P0x0 = np.array([[1, 0],[0, 0]])

        # Define projection operator |1><1|

        P1x1 = np.array([[0, 0],[0, 1]])
        
        # initialize CU operator projector components
        
        # |0><0| component
        if target_qubits[0] == 0:
            O_0 = P0x0
        elif target_qubits[1] == 0:
            O_0 = I
        else:
            O_0 = I
    
        for i in range(1,total_qubits):
            if target_qubits[0] == i:
                O_0 = np.kron(O_0,P0x0)
            elif target_qubits[1] == i:
                O_0 = np.kron(O_0,I)
            else:
                O_0 = np.kron(O_0,I)
                
        O = O_0
        
        # |1><1| component
        if target_qubits[0] == 0:
            O_1 = P1x1
        elif target_qubits[1] == 0:
            O_1 = unitary
        else:
            O_1 = I
    
        for i in range(1,total_qubits):
            if target_qubits[0] == i:
                O_1 = np.kron(O_1,P1x1)
            elif target_qubits[1] == i:
                O_1 = np.kron(O_1,unitary)
            else:
                O_1 = np.kron(O_1,I)
        
        # complete operator
        O = O + O_1
        
    else:
        # unitary gate
        
        # initialize operator
        if target_qubits[0] == 0:
            O = unitary        
        else:
            O = I
            
        for i in range(1,total_qubits):
            if target_qubits[0] == i:
                O = np.kron(O,unitary)
            else:
                O = np.kron(O,I)
    
    return O


In [45]:
def run_program(total_qubits,initial_state, program):
    # read program, and for each gate:
    #   - calculate matrix operator
    #   - multiply state with operator
    # returns final state
    
    final_state = initial_state
    
    for gate in program:
        
        # generating 2**total_qubit x 2**total_qubit operator 
        opr = get_operator(total_qubits, np.array(gate["unitary"]), np.array(gate["target"]))
        
        # updating state
        final_state = np.dot(opr, final_state)
                                      
            
    return final_state

In [46]:
def measure_all(state_vector, num_shots):
    # choose element from state_vector using weighted random and return it's index
    # returns an index array of selected states with length num_shots
    
    # probabilities for the state_vector
    probability = np.square(np.abs(state_vector))            
    
    # numpy.random.choice requires probability sum to be 1.0
    probability = probability/np.sum(probability) # further normalize
    float_err = 1.0 - np.sum(probability) # due to limitations of the float, sum does not add up to be 1.0 within the required tolerance
    
    # add the error to max probability to have the least impact
    max_idx = np.argmax(probability)    
    probability[max_idx] += float_err    
    
    # 1 x num_shots array of state choices
    measurements = np.random.choice(state_vector.size, num_shots, p=probability.flatten())
    
    return measurements

In [47]:
def get_counts(total_qubits, state_vector, num_shots):
    # returns object with statistics in following form:
    #   {
    #      state: number_of_ocurrences,
    #      state: number_of_ocurrences,
    #      state: number_of_ocurrences,
    #      ...
    #   }
    # (only for elements which occoured - returned from measure_all)
    
    # measure num_shots times
    measurements = measure_all(state_vector, num_shots)
    
    # count no. of times each state was measured
    unique, counts = np.unique(measurements, return_counts=True)
    
    # convert index to big endian qubit state 
    u_bin = np.empty(0, dtype=str)    
    for i in range(unique.size):        
        u_bin = np.append(u_bin, np.binary_repr(unique[i], width=total_qubits))
    
    return dict(zip(u_bin, counts))

In [48]:
total_qubits = 8 # can simulate upto 14 qubits with 16GB memory


# gates can be in the form of any single qubit unitary or CU type gate

cct = [
  { "unitary": [[0.70710678, 0.70710678], [0.70710678, -0.70710678]], "target": [0] }, 
  { "unitary": [ [1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 0, 1], [0, 0, 1, 0] ], "target": [0, 1] }
]

# initial state (ground state or any arbitary state)
g_state = get_ground_state(total_qubits)

# update the state vector based on the cicuit and initial state
final_state = run_program(total_qubits,g_state, cct)

# mesurement on the state vector
counts = get_counts(total_qubits,final_state, 1000)

print(counts)

{'00000000': 505, '11000000': 495}
